In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from db import SessionLocal
from sqlalchemy.exc import IntegrityError


In [2]:
## läser vi in våra två csv filer 
df_sebank_customers_1 = pd.read_csv("/Users/k/Documents/TUC/DataQuality/python-bank-project-start/data/sebank_customers_with_accounts.csv")
df_sebank_customers_2 = pd.read_csv("/Users/k/Documents/TUC/DataQuality/python-bank-project-start/data/sebank_customers_with_accounts_2.csv")


In [3]:
merged_df = pd.concat([df_sebank_customers_1, df_sebank_customers_2], ignore_index=True) # slår ihop två dataframe filer med pd.concat()
merged_df.to_csv("sebank_merged_data.csv", index=False) # sparar den till en csv kallad sebank_merged_data.csv

In [4]:
df_sebank_merged = pd.read_csv("/Users/k/Documents/TUC/DataQuality/python-bank-project-start/data/sebank_merged_data.csv") # läser vi in den sammanfogade filen
session = SessionLocal() # skapar en ny session för vår databas där vi nu kan med olika databasoperationer intregerara

In [5]:
# här skapar vi en dataframe för våra unika kunder där vi väljer specifika kolumner och även atr bort dibletter från den mergeade filen
df_customers = df_sebank_merged[['Customer', 'Address', 'Phone', 'Personnummer']].drop_duplicates().reset_index(drop=True)
df_customers.rename(columns={ # vi ändrar kolumnanmen för att det ska passa vår normalisering 
    'Customer': 'name',
    'Address': 'address',
    'Phone': 'phone',
    'Personnummer': 'personnummer'
}, inplace=True)

# skapar en dataframe som innehåller bankkonton med deras koppling till personnummer, här väljer vi BankAccount och Personnummer för vår df
df_accounts = df_sebank_merged[['BankAccount', 'Personnummer']].copy()
df_accounts.rename(columns={
    'BankAccount': 'number', # ändrar BankAccount till number 
    'Personnummer': 'personnummer' # ändrar Personnummer till personnummer 
}, inplace=True)

# Spara normaliserad data som nya CSV-filer
df_customers.to_csv("normalized_customers.csv", index=False)
df_accounts.to_csv("normalized_accounts.csv", index=False)

# denna del är endast för att kunnna få bekräftelse på det vi gjort 
print("Normalisering klar! Filen innehåller:") 
print(f"{len(df_customers)} unika kunder")
print(f"{len(df_accounts)} konton kopplade till kunder")


Normalisering klar! Filen innehåller:
1160 unika kunder
2000 konton kopplade till kunder
